In [5]:
import warnings
import seaborn as sns
import matplotlib.pylab as plt
import PIL
import pandas as pd
import os
from PIL import Image

import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import numpy as np
from numpy.random import seed

Using TensorFlow backend.


In [6]:
#!pip install git+https://github.com/qubvel/efficientnet
from efficientnet import EfficientNetB3

In [7]:
os.listdir('../')

['model', '.ipynb_checkpoints', 'input', 'src', 'kernels']

In [8]:
os.listdir('../input')

['class.csv',
 'train_crop_299',
 'sample_submission.csv',
 '.ipynb_checkpoints',
 'train.csv',
 'test.csv',
 'test_crop_299']

In [9]:
pd.read_csv('../input/train.csv').head(3)

,img_file,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class
0,train_00001.jpg,1,80,641,461,108
1,train_00002.jpg,57,53,293,236,71
2,train_00003.jpg,35,42,227,160,76


In [10]:
SEED = 2
seed(SEED)
tf.set_random_seed(SEED)
%matplotlib inline

In [11]:
#image data normalization
def basic_preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [12]:
#https://github.com/yu4u/cutout-random-erasing
#image augmentation - randombox
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.25, r_1=0.3,
                      r_2=1/0.3, v_l=0, v_h=255, pixel_level=True, ismixup=False):
    def eraser(input_img, ismixup=ismixup):
        if not ismixup:
            input_img = basic_preprocess_input(input_img)
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()
        
        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w*1.3 <= img_w and top + h*1.2 <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [13]:
#image augmentation - mixup
class MixupImageDataGenerator():
    def __init__(self, generator, dataframe, directory, target, batch_size, image_size, israndombox,
                 alpha=0.2, subset=None, scale='rgb',min_mixup=95, max_mixup=100, x_col='img_file'):
        """Constructor for mixup image data generator.

        Arguments:
            generator {object} -- An instance of Keras ImageDataGenerator.
            directory {str} -- Image directory.
            batch_size {int} -- Batch size.
            img_height {int} -- Image height in pixels.
            img_width {int} -- Image width in pixels.

        Keyword Arguments:
            alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
            subset {str} -- 'training' or 'validation' if validation_split is specified in
            `generator` (ImageDataGenerator).(default: {None})
        """

        self.batch_index = 0
        self.batch_size = batch_size
        self.alpha = alpha
        self.min_mixup=min_mixup
        self.max_mixup=max_mixup
        self.israndombox = israndombox
        # First iterator yielding tuples of (x, y)
        self.erasor = get_random_eraser(ismixup=True)
        
        self.generator1 = generator.flow_from_dataframe(dataframe=dataframe, 
                                                        directory=directory,
                                                        x_col = x_col,
                                                        y_col = target,
                                                        target_size=(image_size, image_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        seed=3,
                                                        color_mode=scale,
                                                        shuffle=True)

        # Second iterator yielding tuples of (x, y)
        self.generator2 = generator.flow_from_dataframe(dataframe=dataframe, 
                                                        directory=directory,
                                                        x_col = x_col,
                                                        y_col = target,
                                                        target_size=(image_size, image_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        seed=np.random.randint(100),
                                                        color_mode=scale,
                                                        shuffle=True)


        # Number of images across all classes in image directory.
        self.n = self.generator1.samples

    def reset_index(self):
        """Reset the generator indexes array.
        """

        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        # round up
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        """Get number of steps per epoch based on batch size and
        number of images.

        Returns:
            int -- steps per epoch.
        """
        if (num_samples % batch_size) > 0:
            return (num_samples // batch_size) + 1
        else:
            return num_samples // batch_size

    def __next__(self):
        """Get next batch input/output pair.

        Returns:
            tuple -- batch of input/output pair, (inputs, outputs).
        """

        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0
        
        # Get a pair of inputs and outputs from two iterators.
        X1, y1 = self.generator1.next()
        X2, y2 = self.generator2.next()
        size = X1.shape[0]
        # random sample the lambda value from beta distribution.
        l = np.random.randint(self.min_mixup,self. max_mixup, size=size) / 100
        X_l = l.reshape(size, 1, 1, 1)
        y_l = l.reshape(size, 1)

        # Perform the mixup.
        X = X1 * X_l + X2 * (1 - X_l)
        if self.israndombox:
            for idx in range(size):
                X[idx] = self.erasor(X[idx]) 
        y = y1 * y_l + y2 * (1 - y_l)
        
        return X, y

    def __iter__(self):
        while True:
            yield next(self)


In [14]:
#get data generator
def get_generator(train_df, val_df, test_df,
                  train_datagen,
                  valid_datagen,
                  test_datagen,
                  image_size,
                  ismixup=True,
                  israndombox=True,
                  batch_size=16,
                  valid_batch_size=16,
                  scale='rgb',
                  target='class',
                  min_mixup=95,
                  max_mixup=100,
                  test_batch_size=32):
    
    train_crop_path = os.path.join(DATA_PATH, 'train_crop_{}'.format(image_size))
    test_crop_path = os.path.join(DATA_PATH, 'test_crop_{}'.format(image_size))
    if ismixup:
        train_generator = MixupImageDataGenerator(generator = train_datagen,
                                                    dataframe = train_df,
                                                    directory=train_crop_path,
                                                    batch_size=batch_size,
                                                    image_size=image_size,
                                                    target='class',
                                                    israndombox=israndombox,
                                                    min_mixup=min_mixup,
                                                    max_mixup=max_mixup)
    else:
        train_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df, 
            directory=train_crop_path,
            x_col = 'img_file',
            y_col = target,
            target_size=(image_size, image_size),
            batch_size=batch_size,
            class_mode='categorical',
            seed=3,
            color_mode=scale,
            shuffle=True,
        )

    validation_generator = valid_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=train_crop_path,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size,image_size),
        batch_size=valid_batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale,
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=test_crop_path,
        x_col='img_file',
        y_col=None,
        target_size= (image_size,image_size),
        color_mode=scale,
        class_mode=None,
        batch_size=test_batch_size,
        shuffle=False,
    )
    return train_generator, validation_generator, test_generator

In [15]:
def get_model_path(model_dir, model_name, model_type, fold_step=1):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    from datetime import datetime
    now = datetime.now()
    date_time = now.strftime("%m%d%H%M")
    model_path =model_dir +  "{}_{}_{}_fold_{}.{}".format(model_name, model_type, date_time, fold_step, 'hdf5')
    model_path =model_dir +  "{}_{}_fold_{}.{}".format(model_name, model_type, fold_step, 'hdf5')

    print('>>model path to save: {}'.format(model_path))
    return model_path

In [16]:

class printLearningRate(Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        decay = self.model.optimizer.decay
        iterations = self.model.optimizer.iterations
        lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
        print('Current LR : ', K.eval(lr_with_decay))

In [17]:

def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.

    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.

    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.

    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    
    
    
    
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(keras.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.

    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))


In [18]:
def get_warmup_lr(base_lr, total_count, warmup_epoch=3):
    

    total_steps = int(epoch * total_count / batch_size)
    # Compute the number of warmup batches.
    warmup_steps = int(warmup_epoch * total_count / batch_size)
    # Compute the number of warmup batches.
    warmup_batches = warmup_epoch * total_count / batch_size

    # Create the Learning rate scheduler.
    warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=lr,
                                            total_steps=total_steps,
                                            warmup_learning_rate=lr*0.5,
                                            warmup_steps=warmup_steps,
                                            hold_base_rate_steps=0)
    return warm_up_lr

In [19]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
 

In [20]:
def get_model(model_type, image_size, opt, num_class=196, lr=0.0001, activation='sigmoid',
             dense=512, drop=0.2):
    #'xception', 'vgg19', 'resnet50', 'inception_v3', 'resnet_v2', 'mobilenet_v2', 'inception_resnet_v2'
    if model_type=='xception':
        application = Xception
    elif model_type=='vgg19':
        application = VGG19
    elif model_type=='resnet50':
        application = ResNet50
    elif model_type=='inception_v3':
        application = InceptionV3
    elif model_type=='resnet_v2':
        application = ResNet152V2
    elif model_type=='mobilenet_v2':
        application = MobileNetV2
    elif model_type=='inception_resnet_v2':
        application = InceptionResNetV2
    elif model_type=='EfficientNetB3':
        application = EfficientNetB3
    else:
        application = 'Xception'
        print('>>set application to Xception(Not selected)')
    base_model = application(weights='imagenet', input_shape=(image_size,image_size,3), include_top=False)
    #base_model.trainable = False
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(dense, activation=activation, kernel_initializer='he_normal'))
    model.add(layers.Dropout(drop))
    model.add(layers.Dense(num_class, activation='softmax', kernel_initializer='lecun_normal'))
    #model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc', f1_m])

    return model

In [21]:
def get_callback(patient, model_path, lr, total_count, cosine, min_lr=0.00001, warmup_epoch=10):
    #learning rate scheduler
    if cosine:
        #with cosine decay
        warmup = get_warmup_lr(lr, total_count)
    else:
        #with Palateau
        warmup = ReduceLROnPlateau(monitor = 'val_loss',
                          factor = 0.5, patience = patient / 4,
                          min_lr=min_lr, verbose=1, mode='min',
                                  warmup_epoch=warmup_epoch)
    callbacks = [
        EarlyStopping(monitor='val_f1_m',
                      patience=patient,
                      mode='max',
                      verbose=1),
        ModelCheckpoint(filepath=model_path,
                        monitor='val_f1_m',
                        verbose=1,
                        save_best_only=True,
                        mode='max'),
        
        warmup
    ]
    return callbacks

In [22]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    

In [23]:
#get generator with augmentation
def get_datagen(ismixup, israndombox):
    if israndombox and not ismixup:
        train_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            vertical_flip=False,
            zoom_range=0.2,
            #shear_range=0.5,
            #brightness_range=[0.5, 1.5],
            preprocessing_function=get_random_eraser(v_l=0, v_h=1))  
    
    else:
        train_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            vertical_flip=False,
            zoom_range=0.2,
            #shear_range=0.5,
            #brightness_range=[0.5, 1.5],
            preprocessing_function=basic_preprocess_input
        )
    
    valid_datagen = ImageDataGenerator(
        preprocessing_function=basic_preprocess_input)
    
    test_datagen = ImageDataGenerator(
        preprocessing_function=basic_preprocess_input)
    
    return train_datagen, valid_datagen, test_datagen

In [24]:
def plot_history(history, model_name):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title(model_name)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.ylim(0.0, 1.0)
    plt.grid(color='b', linestyle='-', linewidth=0.1, which='both')
    plt.yticks([x/100 for x in range(0, 100,5)])
    plt.xticks([x for x in range(0, 60,5)])
    plt.show()

In [25]:
def get_image_size(model_type):
    if model_type in ['xception', 'inception_v3', 'inception_resnet_v2','EfficientNetB3']:
        return 299
    elif model_type in ['vgg19', 'resnet50', 'resnet_v2', 'mobilenet_v2']:
        return 224

In [26]:
params = {'xception':{'activation':'sigmoid',
                     'dense':512,
                     'drop':0.25
                     },
          'vgg19':{'activation':'relu',
                                'dense':2048,
                                'drop':0.25},
          'resnet50':{'activation':'relu',
                                'dense':2048,
                                'drop':0.25},
          'inception_v3':{'activation':'relu',
                                'dense':2048,
                                'drop':0.25},
          'inception_resnet_v2':{'activation':'relu',
                                'dense':1024,
                                'drop':0.5},
          'resnet_v2':{'activation':'relu',
                                'dense':2048,
                                'drop':0.25},
          'mobilenet_v2':{'activation':'relu',
                                'dense':2048,
                                'drop':0.25},
          'EfficientNetB3':{'activation':'relu',
                                'dense':2048,
                                'drop':0.25}
         }

In [27]:
#get optimizer
def get_opt(opt, lr=0.0001):
    if opt=='sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.2, nesterov=True)
    elif opt=='nadam':
        return optimizers.Nadam(lr=lr)
    elif opt=='rmsprop':
        return optimizers.RMSprop(lr=lr)

In [28]:
def train(model_type, skf_seed, opt, n_splits=4):
    rs_models = {}
    rs_histories = {}
    
    K.clear_session()
    print('[*]-------------------Start train {} model-------------------'.format(model_type))
    #data split -> kfold
    skf = StratifiedKFold(n_splits=n_splits, random_state=skf_seed)
    
    for fold_step, (train_index, valid_index) in enumerate(skf.split(df_train['img_file'], df_train['class'])):
        print('[*]-------------------{} model_foldstep_{}-------------------'.format(model_type, fold_step))
        #get image size
        image_size = get_image_size(model_type)
        print('>>image_size: {}'.format(image_size))
        #split train / validation set
        X_train = df_train.iloc[train_index, :].reset_index()
        X_val = df_train.iloc[valid_index, :].reset_index()
        #get data generator
        train_gen, valid_gen, test_gen = get_generator(train_df=X_train,
                                                        val_df=X_val,
                                                        test_df=df_test,
                                                        train_datagen = train_datagen,
                                                        valid_datagen = valid_datagen,
                                                        test_datagen= test_datagen,
                                                        image_size=image_size,
                                                        ismixup=True
                                                        )
        #get model path
        model_path = get_model_path(model_dir, model_title, model_type, fold_step+1)
        #get model
        model = get_model(model_type=model_type, image_size=image_size, opt=optimizers.RMSprop(lr=lr), lr=lr,
                          activation=params[model_type]['activation'],
                         dense=params[model_type]['dense'],
                         drop=params[model_type]['drop'])
        #train
        history = model.fit_generator(
            train_gen,
            steps_per_epoch=get_steps(X_train.shape[0], batch_size),
            epochs=epoch,
            validation_data=valid_gen,
            validation_steps=get_steps(X_val.shape[0], valid_batch_size),
            verbose=1,
            callbacks=get_callback(patient, model_path, lr, len(X_train), cosine=False, warmup_epoch=3)
        )
        
        rs_models[model_type+'_foldstep_'+str(fold_step)] = model_path
        rs_histories[model_type+'_foldstep_'+str(fold_step)] = history
    return rs_models, rs_histories

In [29]:
model_dir = '../model/'
DATA_PATH = '../input'
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_train["class"] = df_train["class"].astype('str')
df_test = df_test[['img_file']]
model_title = 'emsemble_final'
mymodels = {}
myhistories = {}

In [30]:
patient = 25
epoch=150
lr=0.0001
batch_size = 16
valid_batch_size = 64
n_splits = 5
model_type = 'EfficientNetB3'

In [ ]:
rs_models, rs_histories = train(model_type = model_type,
                                skf_seed=5,
                                n_splits=n_splits,
                                opt=get_opt('rmsprop', lr=lr))
mymodels[model_type] = rs_models
myhistories[model_type] = rs_histories

[*]-------------------Start train EfficientNetB3 model-------------------
[*]-------------------EfficientNetB3 model_foldstep_0-------------------
>>image_size: 299
>>train_path: ../input/train_crop_299
>>test_path: ../input/test_crop_299
Found 7914 validated image filenames belonging to 196 classes.
Found 7914 validated image filenames belonging to 196 classes.
Found 2076 validated image filenames belonging to 196 classes.
Found 6150 validated image filenames.
>>model path to save: ../model/emsemble_final_EfficientNetB3_fold_1.hdf5
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Epoch 1/150
495/495 [==============================] - 234s 473ms/step - loss: 5.2022 - acc: 0.0160 - f1_m: 0.0000e+00 - val_loss: 4.5868 - val_acc: 0.0739 - val_f1_m: 0.0000e+00

Epoch 00001: val_f1_m improved from -inf to 0.00000, saving model to ../model/emsemble_final_EfficientNetB3_fold_1.hdf5
Epoch 2/150
495/495 [==============================] - 2

Epoch 1/150
497/497 [==============================] - 238s 480ms/step - loss: 5.2299 - acc: 0.0161 - f1_m: 0.0000e+00 - val_loss: 4.7833 - val_acc: 0.0820 - val_f1_m: 0.0000e+00

Epoch 00001: val_f1_m improved from -inf to 0.00000, saving model to ../model/emsemble_final_EfficientNetB3_fold_2.hdf5
Epoch 2/150
497/497 [==============================] - 226s 455ms/step - loss: 4.3101 - acc: 0.1187 - f1_m: 0.0104 - val_loss: 2.8653 - val_acc: 0.3730 - val_f1_m: 0.0746

Epoch 00002: val_f1_m improved from 0.00000 to 0.07455, saving model to ../model/emsemble_final_EfficientNetB3_fold_2.hdf5
Epoch 3/150
497/497 [==============================] - 226s 454ms/step - loss: 3.0834 - acc: 0.3122 - f1_m: 0.1115 - val_loss: 1.7833 - val_acc: 0.5371 - val_f1_m: 0.3912

Epoch 00003: val_f1_m improved from 0.07455 to 0.39116, saving model to ../model/emsemble_final_EfficientNetB3_fold_2.hdf5
Epoch 4/150
497/497 [==============================] - 226s 455ms/step - loss: 2.2956 - acc: 0.4843 - f1_m: 0.

497/497 [==============================] - 226s 455ms/step - loss: 0.3689 - acc: 0.9959 - f1_m: 0.9917 - val_loss: 0.2223 - val_acc: 0.9609 - val_f1_m: 0.9562

Epoch 00066: val_f1_m improved from 0.94522 to 0.95620, saving model to ../model/emsemble_final_EfficientNetB3_fold_2.hdf5
Epoch 67/150
497/497 [==============================] - 226s 455ms/step - loss: 0.3748 - acc: 0.9942 - f1_m: 0.9908 - val_loss: 0.3125 - val_acc: 0.9336 - val_f1_m: 0.9164

Epoch 00067: val_f1_m did not improve from 0.95620
Epoch 68/150
497/497 [==============================] - 226s 455ms/step - loss: 0.3665 - acc: 0.9947 - f1_m: 0.9925 - val_loss: 0.2941 - val_acc: 0.9363 - val_f1_m: 0.9355

Epoch 00068: val_f1_m did not improve from 0.95620
Epoch 69/150
497/497 [==============================] - 226s 455ms/step - loss: 0.3698 - acc: 0.9959 - f1_m: 0.9917 - val_loss: 0.2602 - val_acc: 0.9512 - val_f1_m: 0.9375

Epoch 00069: val_f1_m did not improve from 0.95620
Epoch 70/150
497/497 [=======================

Epoch 7/150
500/500 [==============================] - 228s 457ms/step - loss: 1.3221 - acc: 0.7268 - f1_m: 0.6829 - val_loss: 0.5666 - val_acc: 0.8340 - val_f1_m: 0.8207

Epoch 00007: val_f1_m improved from 0.78321 to 0.82075, saving model to ../model/emsemble_final_EfficientNetB3_fold_3.hdf5
Epoch 8/150
500/500 [==============================] - 228s 456ms/step - loss: 1.1599 - acc: 0.7755 - f1_m: 0.7401 - val_loss: 0.4997 - val_acc: 0.8583 - val_f1_m: 0.8467

Epoch 00008: val_f1_m improved from 0.82075 to 0.84672, saving model to ../model/emsemble_final_EfficientNetB3_fold_3.hdf5
Epoch 9/150
500/500 [==============================] - 228s 456ms/step - loss: 1.0415 - acc: 0.8035 - f1_m: 0.7839 - val_loss: 0.4295 - val_acc: 0.8711 - val_f1_m: 0.8665

Epoch 00009: val_f1_m improved from 0.84672 to 0.86652, saving model to ../model/emsemble_final_EfficientNetB3_fold_3.hdf5
Epoch 10/150
500/500 [==============================] - 228s 456ms/step - loss: 0.9420 - acc: 0.8396 - f1_m: 0.8168


Epoch 00037: val_f1_m did not improve from 0.93655
Epoch 38/150
500/500 [==============================] - 227s 454ms/step - loss: 0.4360 - acc: 0.9850 - f1_m: 0.9790 - val_loss: 0.3172 - val_acc: 0.9258 - val_f1_m: 0.9177

Epoch 00038: val_f1_m did not improve from 0.93655
Epoch 39/150
500/500 [==============================] - 227s 455ms/step - loss: 0.4335 - acc: 0.9834 - f1_m: 0.9783 - val_loss: 0.3516 - val_acc: 0.9258 - val_f1_m: 0.9147

Epoch 00039: val_f1_m did not improve from 0.93655
Epoch 40/150
500/500 [==============================] - 227s 455ms/step - loss: 0.4318 - acc: 0.9841 - f1_m: 0.9792 - val_loss: 0.2927 - val_acc: 0.9261 - val_f1_m: 0.9238

Epoch 00040: val_f1_m did not improve from 0.93655
Epoch 41/150
500/500 [==============================] - 227s 454ms/step - loss: 0.4205 - acc: 0.9881 - f1_m: 0.9834 - val_loss: 0.3124 - val_acc: 0.9277 - val_f1_m: 0.9245

Epoch 00041: val_f1_m did not improve from 0.93655
Epoch 42/150
500/500 [==============================

502/502 [==============================] - 233s 464ms/step - loss: 0.4962 - acc: 0.9690 - f1_m: 0.9620 - val_loss: 0.2975 - val_acc: 0.9254 - val_f1_m: 0.9209

Epoch 00027: val_f1_m improved from 0.91722 to 0.92091, saving model to ../model/emsemble_final_EfficientNetB3_fold_4.hdf5
Epoch 28/150
502/502 [==============================] - 232s 462ms/step - loss: 0.4860 - acc: 0.9709 - f1_m: 0.9639 - val_loss: 0.3537 - val_acc: 0.9261 - val_f1_m: 0.9221

Epoch 00028: val_f1_m improved from 0.92091 to 0.92211, saving model to ../model/emsemble_final_EfficientNetB3_fold_4.hdf5
Epoch 29/150
502/502 [==============================] - 233s 464ms/step - loss: 0.4714 - acc: 0.9781 - f1_m: 0.9680 - val_loss: 0.3570 - val_acc: 0.9093 - val_f1_m: 0.9063

Epoch 00029: val_f1_m did not improve from 0.92211
Epoch 30/150
502/502 [==============================] - 232s 462ms/step - loss: 0.4648 - acc: 0.9772 - f1_m: 0.9695 - val_loss: 0.3506 - val_acc: 0.9153 - val_f1_m: 0.9138

Epoch 00030: val_f1_m di

502/502 [==============================] - 232s 462ms/step - loss: 0.3681 - acc: 0.9955 - f1_m: 0.9930 - val_loss: 0.3322 - val_acc: 0.9199 - val_f1_m: 0.9128

Epoch 00060: val_f1_m did not improve from 0.94593
Epoch 61/150
502/502 [==============================] - 233s 464ms/step - loss: 0.3677 - acc: 0.9954 - f1_m: 0.9927 - val_loss: 0.3173 - val_acc: 0.9294 - val_f1_m: 0.9239

Epoch 00061: val_f1_m did not improve from 0.94593
Epoch 62/150
502/502 [==============================] - 233s 464ms/step - loss: 0.3646 - acc: 0.9966 - f1_m: 0.9941 - val_loss: 0.3514 - val_acc: 0.9395 - val_f1_m: 0.9307

Epoch 00062: val_f1_m did not improve from 0.94593
Epoch 63/150
502/502 [==============================] - 233s 464ms/step - loss: 0.3582 - acc: 0.9960 - f1_m: 0.9935 - val_loss: 0.2742 - val_acc: 0.9415 - val_f1_m: 0.9371

Epoch 00063: val_f1_m did not improve from 0.94593
Epoch 64/150
502/502 [==============================] - 233s 464ms/step - loss: 0.3640 - acc: 0.9948 - f1_m: 0.9918 -

505/505 [==============================] - 232s 459ms/step - loss: 0.4136 - acc: 0.9877 - f1_m: 0.9832 - val_loss: 0.3377 - val_acc: 0.9167 - val_f1_m: 0.9168

Epoch 00041: val_f1_m did not improve from 0.93081
Epoch 42/150
505/505 [==============================] - 232s 459ms/step - loss: 0.4140 - acc: 0.9884 - f1_m: 0.9819 - val_loss: 0.2628 - val_acc: 0.9417 - val_f1_m: 0.9376

Epoch 00042: val_f1_m improved from 0.93081 to 0.93760, saving model to ../model/emsemble_final_EfficientNetB3_fold_5.hdf5
Epoch 43/150
505/505 [==============================] - 232s 460ms/step - loss: 0.4121 - acc: 0.9866 - f1_m: 0.9825 - val_loss: 0.3965 - val_acc: 0.8938 - val_f1_m: 0.8984

Epoch 00043: val_f1_m did not improve from 0.93760
Epoch 44/150
505/505 [==============================] - 232s 460ms/step - loss: 0.4006 - acc: 0.9892 - f1_m: 0.9834 - val_loss: 0.3231 - val_acc: 0.9174 - val_f1_m: 0.9162

Epoch 00044: val_f1_m did not improve from 0.93760
Epoch 45/150
505/505 [=======================

In [ ]:
#fold1 - step 37
#loss: 0.4194 - acc: 0.9879 - f1_m: 0.9828 - val_loss: 0.2951 - val_acc: 0.9394 - val_f1_m: 0.9391
#fold2 - step 66
#loss: 0.3689 - acc: 0.9959 - f1_m: 0.9917 - val_loss: 0.2223 - val_acc: 0.9609 - val_f1_m: 0.9562
#fold3 - step 50
#loss: 0.4076 - acc: 0.9904 - f1_m: 0.9840 - val_loss: 0.2574 - val_acc: 0.9395 - val_f1_m: 0.9458
#fold4 - step 57
#loss: 0.3693 - acc: 0.9950 - f1_m: 0.9916 - val_loss: 0.2222 - val_acc: 0.9556 - val_f1_m: 0.9459
#fold5 - step 55
#loss: 0.3850 - acc: 0.9927 - f1_m: 0.9889 - val_loss: 0.2620 - val_acc: 0.9437 - val_f1_m: 0.9390

In [31]:
model_type = 'EfficientNetB3'
model = get_model(model_type=model_type,
                  image_size=299,
                  opt=get_opt('sgd', lr=lr),
                  lr=lr,
                  activation=params[model_type]['activation'],
                  dense=params[model_type]['dense'],
                  drop=params[model_type]['drop'])

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


In [32]:
train_datagen, valid_datagen, test_datagen = get_datagen(ismixup=True, israndombox=True)

In [33]:
test_batch_size = 32
y_pred = []
for fold_step in range(n_splits):
    predictions = []


    model_path =model_dir +  "{}_{}_fold_{}.{}".format(model_title, model_type, fold_step+1, 'hdf5')
    model.load_weights(model_path)

    train_generator, _, test_generator = get_generator(train_df=df_train,
                                                        val_df=df_train,
                                                        test_df=df_test,
                                                        train_datagen = train_datagen,
                                                        valid_datagen = valid_datagen,
                                                        test_datagen= test_datagen,
                                                        image_size=299,
                                                        test_batch_size=test_batch_size
                                                        )
    test_generator.reset()

    for i in range(10):
        prediction = model.predict_generator(
            generator=test_generator,
            steps = get_steps(df_test.shape[0], test_batch_size),
            verbose=1
        )
        predictions.append(prediction)	
    y_pred.append(predictions)

Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 6150 validated image filenames.
193/193 [==============================] - 40s 206ms/step
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 6150 validated image filenames.
193/193 [==============================] - 39s 204ms/step
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 6150 validated image filenames.
193/193 [==============================] - 40s 208ms/step
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validate

In [34]:
#fold1 - step 37
#loss: 0.4194 - acc: 0.9879 - f1_m: 0.9828 - val_loss: 0.2951 - val_acc: 0.9394 - val_f1_m: 0.9391
#fold2 - step 66
#loss: 0.3689 - acc: 0.9959 - f1_m: 0.9917 - val_loss: 0.2223 - val_acc: 0.9609 - val_f1_m: 0.9562
#fold3 - step 50
#loss: 0.4076 - acc: 0.9904 - f1_m: 0.9840 - val_loss: 0.2574 - val_acc: 0.9395 - val_f1_m: 0.9458
#fold4 - step 57
#loss: 0.3693 - acc: 0.9950 - f1_m: 0.9916 - val_loss: 0.2222 - val_acc: 0.9556 - val_f1_m: 0.9459
#fold5 - step 55
#loss: 0.3850 - acc: 0.9927 - f1_m: 0.9889 - val_loss: 0.2620 - val_acc: 0.9437 - val_f1_m: 0.9390

In [41]:
train_generator, _, test_generator = get_generator(train_df=df_train,
                                                        val_df=df_train,
                                                        test_df=df_test,
                                                        train_datagen = train_datagen,
                                                        valid_datagen = valid_datagen,
                                                        test_datagen= test_datagen,
                                                        image_size=299,
                                                        test_batch_size=32
                                                        )

Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 9990 validated image filenames belonging to 196 classes.
Found 6150 validated image filenames.


In [42]:
y_pred = np.array(y_pred)
y_pred.shape

(5, 10, 6150, 196)

In [43]:
np.save('y_pred_mixup_fold5_tta10', y_pred)

In [44]:
y_pred = np.load('y_pred_mixup_fold5_tta10.npy')

In [45]:
fold_all = y_pred.reshape([-1, 6150, 196])
fold_all = np.max(fold_all, axis=0)
predicted_class_indices=np.argmax(fold_all, axis=1)
labels = (train_generator.generator1.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions_labels = [labels[k] for k in predicted_class_indices]
submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
submission["class"] = predictions_labels
submission.to_csv("submission_{}_fold5_mixup_ensemble_max_tta10_fold_sum.csv".format(model_type), index=False)
submission.head()

,img_file,class
0,test_00001.jpg,124
1,test_00002.jpg,98
2,test_00003.jpg,157
3,test_00004.jpg,94
4,test_00005.jpg,17


In [46]:
fold_2 = y_pred[3,:,:,:]
fold_2 = fold_2.reshape([-1, 6150, 196])
fold_2 = np.max(fold_2, axis=0)
predicted_class_indices=np.argmax(fold_2, axis=1)
labels = (train_generator.generator1.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions_labels = [labels[k] for k in predicted_class_indices]
submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
submission["class"] = predictions_labels
submission.to_csv("submission_{}_fold5_mixup_ensemble_max_tta10_fold_2.csv".format(model_type), index=False)
submission.head()

,img_file,class
0,test_00001.jpg,124
1,test_00002.jpg,98
2,test_00003.jpg,157
3,test_00004.jpg,94
4,test_00005.jpg,18
